## Experiment
- htd on alexnet 

In [1]:
import sys 

EXP_DIR = "/home/fmokadem/NAS/tdcnn/"
sys.path.append(EXP_DIR)

In [2]:
import os
import sys
import torch
from torchvision import models
import time
import random
import numpy as np
import itertools
import math
import argparse
from datetime import datetime
import json
import copy


In [3]:
from common.dataset import load_mnist
from common._logging import setup_logger
from common.utils import (
    count_parameters, 
    measure_inference_time, 
    calculate_accuracy, 
    get_flops, 
    get_conv2d_layers,
    infer_rank, 
    calculate_layer_params,
    replace_conv2d_with_tucker,
    fine_tune
)
from common.load_models import load_model

In [4]:
MODEL_NAME = 'alexnet'
FINETUNE = True
MAX_CFG = 250
ACCU_RQT = .90

In [5]:
MODEL_PATH = os.path.join(EXP_DIR, f'finetuned/saved_models/{MODEL_NAME}_mnist.pth')
LOG_DIR = os.path.join(EXP_DIR, 'logs')
LOG_PREFIX = 'htd_alexnet'

In [6]:
logger = setup_logger(LOG_PREFIX, LOG_DIR, LOG_PREFIX)
logger.info(f"Starting HTD experiment for {MODEL_NAME}")
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

2025-03-29 15:14:19,880 - MainProcess - INFO - Logging to /home/fmokadem/NAS/tdcnn/logs/htd_alexnet_20250329_151419.log
2025-03-29 15:14:19,881 - MainProcess - INFO - Starting HTD experiment for alexnet


In [7]:
# use gpu 0 only
device_idx = 0

if torch.cuda.is_available():
    torch.cuda.set_device(device_idx)
    device = f'cuda:{device_idx}'
else:
    device = "cpu"

logger.info(f"Using device: {device}")

2025-03-29 15:14:21,036 - MainProcess - INFO - Using device: cuda:0


In [8]:
# Load dataset
train_loader, test_loader = load_mnist()
logger.info(f"MNIST loaded: {len(train_loader.dataset)} train, {len(test_loader.dataset)} test samples")

# Load model
model = load_model(MODEL_NAME, MODEL_PATH, device)
logger.info(f"Loaded {MODEL_NAME} from {MODEL_PATH}")

2025-03-29 15:14:22,471 - MainProcess - INFO - MNIST loaded: 60000 train, 10000 test samples
/home/fmokadem/NAS/tdcnn/common/load_models.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experim

In [10]:
train_loader.dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=3)
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [11]:
from torchsummary import summary
summary(model, (3, 244, 244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 60, 60]          23,296
              ReLU-2           [-1, 64, 60, 60]               0
         MaxPool2d-3           [-1, 64, 29, 29]               0
            Conv2d-4          [-1, 192, 29, 29]         307,392
              ReLU-5          [-1, 192, 29, 29]               0
         MaxPool2d-6          [-1, 192, 14, 14]               0
            Conv2d-7          [-1, 384, 14, 14]         663,936
              ReLU-8          [-1, 384, 14, 14]               0
            Conv2d-9          [-1, 256, 14, 14]         884,992
             ReLU-10          [-1, 256, 14, 14]               0
           Conv2d-11          [-1, 256, 14, 14]         590,080
             ReLU-12          [-1, 256, 14, 14]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [12]:
# Extract Conv2D layers
conv_layers = get_conv2d_layers(model)
logger.info(f"Found {len(conv_layers)} Conv2D layers in {MODEL_NAME}")
conv_layers

2025-03-29 15:15:01,078 - MainProcess - INFO - Found 5 Conv2D layers in alexnet


{'features.0': Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)),
 'features.3': Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)),
 'features.6': Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 'features.8': Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 'features.10': Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))}

In [13]:
# Initialize layer information
# Conv2 layers in Pytorch are (Cin, Cout, ks, ks), i.e. a 4D tensor with rank 4, called modes 0 to 3
# we are interested in low rank approximating of modes 0 and 1, i.e. compressing the information in  the channels
# each mode is almost always full rank, i.e. of rank == size
# therefore for the pupose of this exp rank of a layer is the min(Cin, Cout) 


# TODO: complexity ranks in decreasing order layers that are closest to the middle.  
layer_info = {}
for name, layer in conv_layers.items():
    r_i = infer_rank(layer)
    layer_info[name] = {
        'layer': layer,
        'r_i': r_i,
        'params': calculate_layer_params(layer),
        'complexity': None
    }
    logger.info(f"Layer {name}: initial rank R_i = {r_i}, parameters = {layer_info[name]['params']}")

2025-03-29 15:15:02,534 - MainProcess - INFO - Layer features.0: initial rank R_i = 3, parameters = 23232
2025-03-29 15:15:02,535 - MainProcess - INFO - Layer features.3: initial rank R_i = 64, parameters = 307200
2025-03-29 15:15:02,536 - MainProcess - INFO - Layer features.6: initial rank R_i = 192, parameters = 663552
2025-03-29 15:15:02,537 - MainProcess - INFO - Layer features.8: initial rank R_i = 256, parameters = 884736
2025-03-29 15:15:02,538 - MainProcess - INFO - Layer features.10: initial rank R_i = 256, parameters = 589824


In [14]:
layer_info

{'features.0': {'layer': Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)),
  'r_i': 3,
  'params': 23232,
  'complexity': None},
 'features.3': {'layer': Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)),
  'r_i': 64,
  'params': 307200,
  'complexity': None},
 'features.6': {'layer': Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  'r_i': 192,
  'params': 663552,
  'complexity': None},
 'features.8': {'layer': Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  'r_i': 256,
  'params': 884736,
  'complexity': None},
 'features.10': {'layer': Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  'r_i': 256,
  'params': 589824,
  'complexity': None}}

In [15]:
# Compute baseline metrics
baseline_params = count_parameters(model)
baseline_flops = get_flops(model)
baseline_accuracy = calculate_accuracy(model, test_loader, device)
baseline_inference_time = measure_inference_time(model, test_loader, device, num_runs=3)

logger.info(f"Baseline {MODEL_NAME}: params={baseline_params}, "
            f"FLOPs={baseline_flops}, accuracy={baseline_accuracy:.4f}, "
            f"inference_time={baseline_inference_time:.4f}s")

# Timestamp for unique file naming
timestamp = time.strftime("%Y%m%d_%H%M%S")

2025-03-29 15:15:59,238 - MainProcess - INFO - Baseline alexnet: params=57044810, FLOPs=712385098, accuracy=0.9952, inference_time=0.0683s


In [16]:
# Generate possible ranks per layer, that is 1 up to it's rank - 1 
# if layer is of rank 1, then possible ranks are just 1
possible_ranks = {}
for name, info in layer_info.items():
    r_i = info['r_i']
    ranks = [1] + list(range(2, r_i)) 
    possible_ranks[name] = ranks

total_possible_configs = np.prod([max(1, layer_info[name]['r_i'] - 1) for name in layer_info.keys()])
num_configs_to_try = min(total_possible_configs, MAX_CFG)
logger.info(f"Total possible configurations: {total_possible_configs}, will try: {num_configs_to_try}")

2025-03-29 15:16:08,075 - MainProcess - INFO - Total possible configurations: 1564891650, will try: 250


In [17]:
from torch import nn
def construct_layer_dict(model):
    layer_dict = {}
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            W = module.weight.data
            cin = module.in_channels
            cout = module.out_channels
            layer_dict[name] = (W, cin, cout)
    return layer_dict

layer_dict = construct_layer_dict(model)

In [18]:
from tensorly.decomposition import partial_tucker
import tensorly as tl    
from tensorly.tucker_tensor import tucker_to_tensor
from tensorly.metrics.regression import MSE

tl.set_backend('pytorch')

# Heuristic: layers with bigger cin*cout more suseptible to lower ranks 
def get_size_based_probs(cin, cout, rank_candidates, beta=1.0):
    size = cin * cout
    preference = np.log(size + 1)  # +1 to avoid log(0), though unlikely
    scores = [-beta * r / preference for r in rank_candidates]
    probs = np.exp(scores) / np.sum(np.exp(scores))
    return probs

# Heuristic: Layers with higher reconstruction erros are less suseptible to low ranks 
def compute_sensitivity(W, cin, cout):
    rank = [max(1, cout // 2), max(1, cin // 2)]
    (core, factors) = partial_tucker(W, modes=[0, 1], rank=rank, init='svd')
       
    reconstructed_W = tucker_to_tensor(core, factors)
    return MSE(W, reconstructed_W) 

# Precompute sensitivity for all layers
def precompute_sensitivities(layer_dict):
    sensitivities = {}
    for name, (W, cin, cout) in layer_dict.items():
        sensitivities[name] = compute_sensitivity(W, cin, cout).to('cpu')
    # Normalize sensitivities to [0, 1]
    max_error = max(list(sensitivities.values()))
    if max_error > 0:  # Avoid division by zero
        sensitivities = {name: err / max_error for name, err in sensitivities.items()}
    return sensitivities
    
def get_sensitivity_probs(sensitivity, rank_candidates, alpha=1.0):
    max_rank = max(rank_candidates)
    scores = [(r / max_rank) ** (alpha * sensitivity) for r in rank_candidates]
    probs = scores / np.sum(scores)
    return probs

def get_rank_candidates(channels, r=(75, 45, -5)):
    s, f, stride = r
    percentages = np.arange(s, f, stride) / 100
    candidates = set([max(1, int(channels * p)) for p in percentages])
    return candidates

In [19]:
# Generator function to yield num_cfg configurations
def generate_configs(layer_dict, num_cfg):
    # Precompute rank candidates for each layer
    rank_candidates = {}
    for name, (W, cin, cout) in layer_dict.items():
        cin_candidates = get_rank_candidates(cin)  
        cout_candidates = get_rank_candidates(cout)  
        
        rank_pairs = list(itertools.product(cin_candidates, cout_candidates))
        # we sort to keep higher ranks on top of the search 
        # no heuristic, budget num_cfg accross all layers equally 
        # num_cfg / len(layer_dict.keys) searches per layer
        spl = int(num_cfg ** (1 / len(layer_dict.keys())))
        rank_candidates[name] = sorted(rank_pairs, key=lambda x: (-x[0], -x[1]))[:spl]
    
    # Yield exactly num_cfg random configurations
    for config in itertools.product(*rank_candidates.values()):
        yield dict(zip(layer_dict.keys(), config))

In [20]:
def duplicate_model(model):
    # Check if the model name is valid
    model_name = MODEL_NAME
    
    if model_name == 'vgg':
        model_cp = models.vgg16(weights=None)
        model_cp.classifier[6] = nn.Linear(4096, 10)
        
    elif model_name == 'alexnet':
        model_cp = models.alexnet(weights=None)
        model_cp.classifier[6] = nn.Linear(4096, 10)
        
    elif model_name == 'resnet':
        model_cp = models.resnet18(weights=None)
        model_cp.fc = nn.Linear(512, 10)

    model_cp.load_state_dict(model.state_dict())   
    return model_cp

In [21]:
configs = generate_configs(layer_dict, MAX_CFG)
next(configs)

{'features.0': (2, 48),
 'features.3': (48, 144),
 'features.6': (144, 288),
 'features.8': (288, 192),
 'features.10': (192, 192)}

In [22]:
def process_config(config, device_idx):
    try:        
        # Explicitly create a new scope to help with garbage collection
        with torch.enable_grad():

            config_str = ", ".join([f"{k}:{v}" for k, v in config.items()])
            logger.info(f"Compressing to:{config_str}")
            
            # Apply the configuration
            model.to('cpu')
            compressed_model = duplicate_model(model)
            compressed_model.to(device)

            for name, rank in config.items():
                layer = layer_info[name]['layer']
                compressed_model = replace_conv2d_with_tucker(compressed_model, name, layer, rank)

            # verify compressed model is still on gpu
            compressed_model.to(device)

            # Finetune for 3 epochs 
            if FINETUNE: 
                logger.info(f"finetuning:{config_str}")
                compressed_model = fine_tune(compressed_model, train_loader, device, epochs=3, lr=0.001)
                
            # Evaluate the model
            accuracy = calculate_accuracy(compressed_model, test_loader, device)
            params = count_parameters(compressed_model)
            flops = get_flops(compressed_model)
            inference_time = measure_inference_time(compressed_model, test_loader, device, num_runs=3)
            compression_rate = baseline_params / params if params > 0 else float('inf')
            
            result = {
                'config_str': config_str,
                'params': params,
                'flops': flops,
                'accuracy': accuracy,
                'inference_time': inference_time,
                'compression_rate': compression_rate,
                'accepted': True if accuracy >= acceptance_threshold else False
            }
            result_str = json.dumps(result, indent=4, default=str)
            logger.info(f"compressed_model:\n{result_str}")

            return result
    
    except Exception as e:
        logger.error(f"Error processing config: {config}. Error: {str(e)}")
        return None
    
    finally:
        # Explicit cleanup
        if 'compressed_model' in locals():
            del compressed_model
        
        # Clear CUDA cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

In [23]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import uuid


num_workers = 4
num_gpus = 1  # torch.cuda.device_count() if torch.cuda.is_available() else 0
results = []
tried_count = 0
accepted_models = []


configs = generate_configs(layer_dict, MAX_CFG)
acceptance_threshold = ACCU_RQT * baseline_accuracy

with ThreadPoolExecutor(max_workers=num_workers) as executor:
    # Submit tasks with device indices in round-robin
    if num_gpus > 0:
        futures = [executor.submit(process_config, config, 0) for config in configs] #- i % num_gpus) for i, config in enumerate(configs)]
        
        for future in as_completed(futures):
            try:
                result = future.result()
                if result is not None:
                    results.append(result)
                    
                    if result['accepted']:
                        accepted_models.append(result)
                    
                    tried_count += 1
                    if tried_count % 10 == 0:
                        logger.info(f"Evaluated {tried_count} configurations, found {len(accepted_models)} accepted models")
            
            except Exception as e:
                logger.error(f"Error in future processing: {str(e)}")
    else:
        logger.warning('NO GPU')

2025-03-29 15:26:28,829 - MainProcess - INFO - Compressing to:features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288, 192), features.10:(192, 192)
2025-03-29 15:26:28,830 - MainProcess - INFO - Compressing to:features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288, 192), features.10:(192, 179)
2025-03-29 15:26:28,831 - MainProcess - INFO - Compressing to:features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288, 192), features.10:(192, 166)
2025-03-29 15:26:28,832 - MainProcess - INFO - Compressing to:features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288, 179), features.10:(192, 192)
2025-03-29 15:26:36,985 - MainProcess - INFO - finetuning:features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288, 179), features.10:(192, 192)
2025-03-29 15:26:37,122 - MainProcess - INFO - finetuning:features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288,

Epoch 1/3, Loss: 0.3240
Epoch 1/3, Loss: 0.3190
Epoch 1/3, Loss: 0.3190
Epoch 1/3, Loss: 0.3226
Epoch 2/3, Loss: 0.0550
Epoch 2/3, Loss: 0.0522
Epoch 2/3, Loss: 0.0525
Epoch 2/3, Loss: 0.0521
Epoch 3/3, Loss: 0.0393
Epoch 3/3, Loss: 0.0386
Epoch 3/3, Loss: 0.0411
Epoch 3/3, Loss: 0.0400


2025-03-29 15:37:33,150 - MainProcess - INFO - compressed_model:
{
    "config_str": "features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288, 179), features.10:(192, 192)",
    "params": 55985001,
    "flops": 404517440,
    "accuracy": 0.9893,
    "inference_time": 0.2003718105850706,
    "compression_rate": 1.0189302309738282,
    "accepted": true
}
2025-03-29 15:37:33,172 - MainProcess - INFO - Compressing to:features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288, 179), features.10:(192, 179)
2025-03-29 15:37:33,623 - MainProcess - INFO - compressed_model:
{
    "config_str": "features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288, 192), features.10:(192, 179)",
    "params": 55994153,
    "flops": 349793280,
    "accuracy": 0.9879,
    "inference_time": 0.20037696619701995,
    "compression_rate": 1.0187636912732656,
    "accepted": true
}
2025-03-29 15:37:33,627 - MainProcess - INFO - Compressing to:feat

Epoch 1/3, Loss: 0.3255
Epoch 1/3, Loss: 0.3316
Epoch 1/3, Loss: 0.3166
Epoch 1/3, Loss: 0.3213
Epoch 2/3, Loss: 0.0537
Epoch 2/3, Loss: 0.0541
Epoch 2/3, Loss: 0.0526
Epoch 2/3, Loss: 0.0538
Epoch 3/3, Loss: 0.0393
Epoch 3/3, Loss: 0.0398
Epoch 3/3, Loss: 0.0395
Epoch 3/3, Loss: 0.0393


2025-03-29 15:48:34,399 - MainProcess - INFO - compressed_model:
{
    "config_str": "features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288, 179), features.10:(192, 179)",
    "params": 55959209,
    "flops": 343887744,
    "accuracy": 0.9897,
    "inference_time": 0.17473302626559184,
    "compression_rate": 1.0193998632110757,
    "accepted": true
}
2025-03-29 15:48:34,414 - MainProcess - INFO - Compressing to:features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288, 166), features.10:(192, 166)
2025-03-29 15:48:37,465 - MainProcess - INFO - compressed_model:
{
    "config_str": "features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288, 179), features.10:(192, 166)",
    "params": 55933417,
    "flops": 339528896,
    "accuracy": 0.9892,
    "inference_time": 0.17589700348564521,
    "compression_rate": 1.0198699285616681,
    "accepted": true
}
2025-03-29 15:48:37,468 - MainProcess - INFO - Compressing to:fea

...

2025-03-30 00:10:15,384 - MainProcess - INFO - compressed_model:
{
    "config_str": "features.0:(2, 41), features.3:(48, 124), features.6:(144, 288), features.8:(288, 166), features.10:(192, 192)",
    "params": 55950057,
    "flops": 342341056,
    "accuracy": 0.9889,
    "inference_time": 0.1931710643120379,
    "compression_rate": 1.019566610986652,
    "accepted": true
}
2025-03-30 00:10:15,392 - MainProcess - INFO - Compressing to:features.0:(2, 41), features.3:(48, 124), features.6:(144, 268), features.8:(288, 192), features.10:(192, 166)
2025-03-30 00:10:21,767 - MainProcess - INFO - finetuning:features.0:(2, 41), features.3:(48, 124), features.6:(144, 268), features.8:(288, 192), features.10:(192, 166)
2025-03-30 00:10:22,024 - MainProcess - ERROR - Error processing config: {'features.0': (2, 41), 'features.3': (48, 124), 'features.6': (144, 268), 'features.8': (288, 192), 'features.10': (192, 166)}. Error: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total cap

Epoch 1/3, Loss: 0.3198
Epoch 1/3, Loss: 0.3257
Epoch 1/3, Loss: 0.3246
Epoch 1/3, Loss: 0.3207
Epoch 2/3, Loss: 0.0523
Epoch 2/3, Loss: 0.0533
Epoch 2/3, Loss: 0.0525
Epoch 2/3, Loss: 0.0539
Epoch 3/3, Loss: 0.0396
Epoch 3/3, Loss: 0.0396
Epoch 3/3, Loss: 0.0389
Epoch 3/3, Loss: 0.0398


2025-03-30 00:22:12,633 - MainProcess - INFO - compressed_model:
{
    "config_str": "features.0:(2, 41), features.3:(48, 124), features.6:(144, 268), features.8:(288, 166), features.10:(192, 166)",
    "params": 55898473,
    "flops": 333623360,
    "accuracy": 0.9896,
    "inference_time": 0.18406136121972605,
    "compression_rate": 1.0205074832723964,
    "accepted": true
}
2025-03-30 00:22:12,643 - MainProcess - INFO - Compressing to:features.0:(2, 41), features.3:(48, 124), features.6:(144, 249), features.8:(288, 179), features.10:(192, 166)
2025-03-30 00:22:18,289 - MainProcess - INFO - finetuning:features.0:(2, 41), features.3:(48, 124), features.6:(144, 249), features.8:(288, 179), features.10:(192, 166)
2025-03-30 00:22:24,438 - MainProcess - INFO - compressed_model:
{
    "config_str": "features.0:(2, 41), features.3:(48, 124), features.6:(144, 249), features.8:(288, 179), features.10:(192, 179)",
    "params": 55959209,
    "flops": 393506688,
    "accuracy": 0.9887,
    "i

Epoch 1/3, Loss: 0.3255
Epoch 1/3, Loss: 0.3208
Epoch 1/3, Loss: 0.3120
Epoch 1/3, Loss: 0.3221
Epoch 2/3, Loss: 0.0528
Epoch 2/3, Loss: 0.0532
Epoch 2/3, Loss: 0.0536
Epoch 2/3, Loss: 0.0535
Epoch 3/3, Loss: 0.0406
Epoch 3/3, Loss: 0.0396
Epoch 3/3, Loss: 0.0400
Epoch 3/3, Loss: 0.0394


2025-03-30 00:33:41,647 - MainProcess - INFO - compressed_model:
{
    "config_str": "features.0:(2, 41), features.3:(48, 124), features.6:(144, 249), features.8:(288, 179), features.10:(192, 166)",
    "params": 55933417,
    "flops": 339528896,
    "accuracy": 0.9905,
    "inference_time": 0.17579424811523178,
    "compression_rate": 1.0198699285616681,
    "accepted": true
}
2025-03-30 00:33:53,489 - MainProcess - INFO - compressed_model:
{
    "config_str": "features.0:(2, 41), features.3:(48, 124), features.6:(144, 249), features.8:(288, 166), features.10:(192, 192)",
    "params": 55950057,
    "flops": 342341056,
    "accuracy": 0.9903,
    "inference_time": 0.1697689347965702,
    "compression_rate": 1.019566610986652,
    "accepted": true
}
2025-03-30 00:34:09,639 - MainProcess - INFO - compressed_model:
{
    "config_str": "features.0:(2, 41), features.3:(48, 124), features.6:(144, 249), features.8:(288, 166), features.10:(192, 179)",
    "params": 55924265,
    "flops": 3379

In [ ]:
# Save top 10 models
if accepted_models:
    accepted_models.sort(key=lambda x: x['score'], reverse=True)
    top_models = accepted_models[:10]
    logger.info(f"Found {len(accepted_models)} accepted models, saving top {len(top_models)}")
    
    for model_info in top_models:
        final_save_path = os.path.join(save_dir, f"htd_{MODEL_NAME}_{model_info['config_str']}_{timestamp}.pth")
        os.rename(model_info['model_path'], final_save_path)
        logger.info(f"Saved top model {model_info['config_str']} to {final_save_path}")/home/fmokadem/NAS/tdcnn/README.md
    
    # Clean up remaining temporary files
    for model_info in accepted_models[10:]:
        if model_info['model_path'] and os.path.exists(model_info['model_path']):
            os.remove(model_info['model_path'])
else:
    logger.info("No accepted models found")

# Clean up temp directory if empty
if not os.listdir(temp_dir):
    os.rmdir(temp_dir)

logger.info(f"HTD experiment for {MODEL_NAME} completed")

In [30]:
max(results, key = lambda x: x['compression_rate'])

{'config_str': 'features.0:(2, 48), features.3:(48, 144), features.6:(144, 288), features.8:(288, 166), features.10:(192, 166)',
 'params': 55898473,
 'flops': 333623360,
 'accuracy': 0.9906,
 'inference_time': np.float64(0.1705096041320995),
 'compression_rate': 1.0205074832723964,
 'accepted': True}